# TFB IA & BigData XII _ KeepCoding

## CRAWLER UNVOTES

A continuación presentamos nuestro trabajo en este área de recopilación del dato. Tras determinar la fuente de datos con la trajaremos en nuestro proyecto, se ha determinado la siguiente url como objetivo de scraping:

https://digitallibrary.un.org/search?ln=en&cc=Voting+Data&p=&f=&action_search=Search&rm=&sf=&so=d&rg=50&c=Voting+Data&c=&of=hb&fti=0&fti=0

### Una breve explicación sobre este proceso de extracción de datos

En primer lugar, usaremos una biblioteca de python llamada selenium debido al prestigio en su labor de interactuar con las páginas web (haciendo click, por ejemplo) gracias a su webdriver (navegador controlado por el script), en este caso usaremos firefox.

Importamos la libreria anteriormente descrita:

In [ ]:
pip install selenium

El siguiente código tiene como objetivo extraer todas las url contenidas en los hrefs de nuestro interés para posteriormente almacenarlas en un array y escribirlas en disco (archivo ./href) para su ulterior procesamiento individual en tanto captura de los datos objetivos que nos interesan para nuestro análisis.

Una explicación breve de este código consistiría en señalar cómo el bot irá obteniendo los hrefs que contienen las url objetivo y una vez completada esa página, hace click en el botón siguiente para repetir el proceso con la siguiente página. Este proceso (el bucle) se dentendrá cuando la url anterior coincida con la recien url visitada tras pulsar el botón de siguiente: no hay más páginas.

In [ ]:


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def parse_url(url):
   
    driver = webdriver.Firefox()
    driver.get(url)
   
    final = []
    next_bool  = True
    while next_bool:
        elem = driver.find_elements(By.CLASS_NAME, "result-title")
       
        for e in elem:
            p = e.find_element(By.TAG_NAME, "a")
            final.append(p.get_attribute("href"))
            print("Resultado final - conjunto de URLs objetivo -"
            print(final)
       
        try:
            url_after = driver.current_url
            next_  = driver.find_element(By.CLASS_NAME,"rec-navigation")
            next_1 = next_.find_elements(By.TAG_NAME ,"a")
            next_1[len(next_1)-1].click()

            url_before = driver.current_url
            if url_after == url_before:
                next_bool = False
            else:
                next_bool = True

        except:
            next_bool = False
        
    driver.close()
    return final
 
url = "https://digitallibrary.un.org/search?ln=en&cc=Voting+Data&p=&f=&action_search=Search&rm=&sf=&so=d&rg=50&c=Voting+Data&c=&of=hb&fti=0&fti=0"


parsed_tree = parse_url(url)


if parsed_tree is not None:
    file = open("hrefs","w",encoding="utf-8")
    file.write(str(parsed_tree))
    file.close()
    print("File printed")
    
    


La celda inmediatamente inferior está pensada para ahorrar costes computacionales y de tiempo de cara a una presentación en video de este proceso, limitando el número de url extraidas que va a visitar el siguiente script para la captura y guardado de la porción de html donde se encuentra la información que nos interesa

In [ ]:
break_ = 7 # Cantidad de urls objetivo de las que vamos a extraer información con fines de abreviar un proceso muy largo.

In [ ]:
El script que se muestra en la celda siguiente tiene la función ya comentada: visitar cada url anteriormente extraída y guardar en disco (./dowloads) una porción de html donde se encuentra la información que nos interesa

Para ello, usa el webdriver de firefox por cada url y selecciona una porción de html que guardará en disco mientras informa de cómo está resultado todo el procedemiento mediante la salida por consola

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import ast
import os

def parse_url(url):

    os.makedirs("dowloads", exist_ok=True)
    
    driver = webdriver.Firefox()
    driver.get(url)
    result = driver.find_element(By.ID,"details-collapse")
    result = result.get_attribute("innerHTML")
    result = result + ""
   
    driver.close()
    return result
   
        
# def start():

f   = open("hrefs", mode="r", encoding="utf-8")
url = f.read()
url = ast.literal_eval(url)
numurl = len(url)
print("Cantidad de urls: "+str(numurl))

count = 0
for u in url:
    if break_ == count:
        print("fin del procedimiento abreviado")
        break
    count += 1
    parsed_tree = parse_url(u)
    
    if parsed_tree is not None:
        i = url.index(u)
        file = open("dowloads/contents_"+str(i),"w",encoding="utf-8")
        file.write(str(parsed_tree))
        file.close()
        if i%10==0:
            print("File printed -- "+str(i+1)+" elementos de "+str(numurl)+" totales")
        else:
            print("File printed")
print("The End")


In [ ]:
Una vez finalizado el proceso debemos disponer de las porciones de código extraídos en la carpeta dowloads

In [ ]:
A continuación, realizaremos un procesamiento de las porciones de html extraidas, imprimiento por consola todos los datos probablemente revelevantes para nuestro análisis

In [ ]:
Usaremos la biblioteca lxml de python que nos permite operar con el DOM tanto de XML como de HTML

In [ ]:
pip install lxml

In [ ]:
import lxml
from lxml import etree
from lxml import html

# file = open("dowloads/contents_0")
arr = os.listdir('dowloads')
for file in arr:
    file = open("dowloads/"+file)
    txt  = file.read()
    file.close()
    parser = etree.HTMLParser()
    t=etree.fromstring(txt,parser)
    count = -1
    divs  = []
    for e in t.iter():
        count = count + 1 
        if e.tag == "div":
            divs.append(e)

    notVotes = []
    print(len(divs))
    count = -1
    for e in divs:
        count = count + 1
        try:
         match count:
                case 0:
                    print("---------------------------title")
                    print(e.getchildren()[1].text)
                    print(count)   
                case 1:
                    print("---------------------------Agenda")
                    print(e.getchildren()[1].getchildren()[0].text) 
                    print(count)   
                case 2:
                    print("---------------------------resolution")
                    print(e.getchildren()[1].getchildren()[0].text) 
                    print(count)   
                case 7:
                    print("---------------------------Summary")
                    print(e.getchildren()[1].getchildren()[1].text) 
                    print(count)   
                case 10:
                    print("---------------------------DATE")
                    print(e.getchildren()[1].text) 
                case 11:
                    try:
                        print("---------------------------Vote")
                        data = html.tostring(e.getchildren()[1], method='text', encoding='unicode')
                        print(data) 
                    except:
                        notVotes.append("file") # guardamos aquellas resoluciones que no tienen aún votos para su exclusión posterior   
        except:
            print("hubo algún campo vacío:")
               



In [ ]:
Tras este procedimiento, estamos en disposción de dar estructura a los datos según los requerimientos del análisis de datos, bien sea en JSON, CSV o cualquier otro conveniente.